In [6]:
import nibabel as nib
import numpy as np
import glob

# Liste des fichiers NIfTI à fusionner
file_list = glob.glob(r'/NAS/tupac/protocoles/PULSE/DTI/FA/*_onMNI.nii.gz')

# Initialiser une variable pour accumuler la somme des images
sum_image = None

# Lire chaque fichier et accumuler la somme
for file in file_list:
    img = nib.load(file)
    img_data = img.get_fdata()

    if sum_image is None:
        sum_image = np.zeros_like(img_data, dtype=np.float32)

    sum_image += img_data

# Calculer la moyenne
mean_image_data = sum_image / len(file_list)

# Créer une nouvelle image NIfTI pour la moyenne
mean_img = nib.Nifti1Image(mean_image_data, img.affine, img.header)

# Sauvegarder l'image moyenne
nib.save(mean_img, r'/NAS/tupac/protocoles/PULSE/DTI/FA/mean_image.nii.gz')


In [13]:
import nibabel as nib
import numpy as np
import glob

# Liste des fichiers NIfTI à fusionner
file_list = glob.glob(r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/template_PAM50/sub-*/ses-*/sub-*_ses-M*_dwi_fa_on_PAM50.nii.gz')

# Initialiser une variable pour accumuler la somme des images
sum_image = None

# Lire chaque fichier et accumuler la somme
for file in file_list:
    img = nib.load(file)
    img_data = img.get_fdata()

    if sum_image is None:
        sum_image = np.zeros_like(img_data, dtype=np.float32)

    sum_image += img_data

# Calculer la moyenne
mean_image_data = sum_image / len(file_list)

# Créer une nouvelle image NIfTI pour la moyenne
mean_img = nib.Nifti1Image(mean_image_data, img.affine, img.header)

# Sauvegarder l'image moyenne
nib.save(mean_img, r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/mean_image_FA_onPAM50.nii.gz')


In [10]:
file_list

[]

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
import matplotlib.pyplot as plt

# # Lire les fichiers avec nibabel
# #images = [ for file_path in file_paths]
# # Soustraire l'image '801T2W_SPAIR2D_T2w_masklesioncervicalcord_dilated' de 'seg_cerv_rpi_labeled_eroded'
# dilated_les_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/sub-012_ses-M6_acq-801T2W_SPAIR2D_T2w_masklesioncervicalcord_dilated.nii.gz').get_fdata()
# eroded_les_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/sub-012_ses-M6_acq-801T2W_SPAIR2D_T2w_masklesioncervicalcord_eroded.nii.gz').get_fdata()
# segmented_vert_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/seg_cerv_rpi_labeled_eroded.nii.gz').get_fdata()


# # Ne garder que les labels 3 à 6 dans dilated_img
# filtered_vert_data = np.where((segmented_vert_M6 >= 3) & (segmented_vert_M6 <= 6), 1, 0)
# result_vertminusles_data = filtered_vert_data - dilated_les_M6 - dilated_les_M3

# Chemin des fichiers à récupérer
#file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_NAWM_mean.txt'
file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_M3onM6.nii.gz'

# Récupérer les fichiers avec une taille supérieure à 0 octet
valid_files = [file for file in glob.glob(file_pattern) if os.path.getsize(file) > 0]

# Afficher les fichiers récupérés
print("Fichiers valides :", valid_files)
# Créer un dictionnaire pour stocker les données
data = {}


# Afficher des informations sur chaque fichier chargé
for i, img in enumerate(valid_files[:]):    
    SUBJ = img.split('/')[8]
    print(f"Traitement du fichier ${SUBJ} {i+1}/{len(valid_files[:])}: {img}")
    DWI_quant = nib.load(img).get_fdata()
    # Calculer la moyenne de DWI_quant pour les voxels dans filtered_vert_data
    nawm_name = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJ}/mask_NAWM_M6_eroded_3_6.nii.gz'
    lesion_name = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJ}/mask_lesion_M3_eroded_3_6.nii.gz'
    print(f"Vérification des fichiers : {nawm_name}, {lesion_name}")
    if os.path.exists(nawm_name) and os.path.exists(lesion_name):
        nawm_data = nib.load(nawm_name).get_fdata()
        mean_nawm = np.mean(DWI_quant[(nawm_data == 1) & (DWI_quant != 0)])
        
        filtered_vert_data = nib.load(lesion_name).get_fdata()    
        # Calculer la moyenne de DWI_quant pour les voxels dans result_vertminusles_data
        print(np.sum(filtered_vert_data))
        if np.sum(filtered_vert_data) == 0:
            mean_lesion = 0
        else:
            mean_lesion = np.mean(DWI_quant[(filtered_vert_data == 1) & (DWI_quant != 0)])

        print(f"Moyenne pour NAWM : {mean_nawm}")
        print(f"Moyenne pour lesion : {mean_lesion}")

# # Afficher l'histogramme
# plt.hist(DWI_quant[filtered_vert_data == 1], bins=50, color='blue', alpha=0.7)
# plt.title("Histogramme des valeurs de DWI_quant pour NAWM")
# plt.xlabel("Valeurs de DWI_quant")
# plt.ylabel("Fréquence")
# plt.show()

    
    


Fichiers valides : ['/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-078/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-107/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-048/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-098/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/der

In [40]:
import glob
import os
import pandas as pd

# Chemin des fichiers à récupérer
file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_NAWM_mean.txt'

# Récupérer les fichiers avec une taille supérieure à 0 octet
valid_files = [file for file in glob.glob(file_pattern) if os.path.getsize(file) > 0]

# Afficher les fichiers récupérés
print("Fichiers valides :", valid_files)
# Créer un dictionnaire pour stocker les données
data = {}

# Parcourir les fichiers valides
for file in valid_files:
    # Extraire la clé "sub-..." du chemin
    key = os.path.basename(os.path.dirname(file))
    
    # Lire la valeur FA_NAWM contenue dans le fichier
    value = float(open(file, 'r').read().strip())
    
    # Ajouter la clé et la valeur au dictionnaire
    #dta[key] = value

    # Chemin du fichier fa_lesion correspondant
    lesion_file = os.path.join(os.path.dirname(file), 'fa_lesion_M6_mean.txt')
    
    # Vérifier si le fichier fa_lesion existe et a une taille > 0
    if os.path.exists(lesion_file) and os.path.getsize(lesion_file) > 0:
        lesion_value = float(open(lesion_file, 'r').read().strip())
    else:
        lesion_value = 0  # Valeur par défaut si le fichier n'existe pas ou est vide
    
    # Ajouter la valeur fa_lesion au dictionnaire
    if value > 0 and lesion_value > 0:
        data[key] = {'FA_NAWM': value, 'FA_lesion': lesion_value}


# Créer un DataFrame à partir du dictionnaire
df_FA_NAWM = pd.DataFrame.from_dict(data, orient='index').reset_index()
df_FA_NAWM.rename(columns={'index': 'Subject'}, inplace=True)

# Afficher le DataFrame
print(df_FA_NAWM)

Fichiers valides : ['/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-078/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-107/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-098/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-111/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/der

In [73]:
import nibabel as nib
import os
import pandas as pd

# Initialiser une liste pour stocker les résultats
results = []
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    # Chemins des fichiers pour le sujet
    nawm_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/NAWM.nii.gz'
    mask_lesion_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/mask_lesion_M6_eroded_3_6.nii.gz'
    print(f'pastient {subj}')
    # Vérifier si les trois fichiers existent
    if os.path.exists(nawm_path) and os.path.exists(mask_lesion_path) and os.path.exists(fa_path):
        # Charger les images
        nawm_img = nib.load(nawm_path)
        mask_lesion_img = nib.load(mask_lesion_path)
        fa_img = nib.load(fa_path)

        # Extraire les données des images
        nawm_data = nawm_img.get_fdata()
        mask_lesion_data = mask_lesion_img.get_fdata()
        fa_data = fa_img.get_fdata()

        # Calculer les moyennes
        mean_fa_nawm = np.nanmean(fa_data[(nawm_data == 1) & (fa_data != 0)])
        mean_fa_lesion = np.nanmean(fa_data[(mask_lesion_data == 1) & (fa_data != 0)])
        print(f'Mean FA_NAWM for {subj}: {mean_fa_nawm}')
        print(f'Mean FA_lesion for {subj}: {mean_fa_lesion}')
        # Ajouter les résultats à la liste
        results.append({'Subject': subj, 'FA_NAWM': mean_fa_nawm, 'FA_lesion': mean_fa_lesion})

# Créer un DataFrame à partir des résultats
df_results = pd.DataFrame(results)

# Afficher le DataFrame
print(df_results)


pastient sub-003
pastient sub-004
Mean FA_NAWM for sub-004: 0.2946834192423773
Mean FA_lesion for sub-004: 0.3154811413403143
pastient sub-005


/tmp/ipykernel_2170584/3174413930.py:31: RuntimeWarning:

Mean of empty slice



Mean FA_NAWM for sub-005: 0.39293237328151154
Mean FA_lesion for sub-005: nan
pastient sub-006
Mean FA_NAWM for sub-006: 0.3641644510144149
Mean FA_lesion for sub-006: 0.5885803055182193
pastient sub-007
pastient sub-008
Mean FA_NAWM for sub-008: 0.267661564237411
Mean FA_lesion for sub-008: 0.22108110248065385
pastient sub-009
Mean FA_NAWM for sub-009: 0.3982683434662422
Mean FA_lesion for sub-009: nan
pastient sub-010
Mean FA_NAWM for sub-010: 0.24443359330122358
Mean FA_lesion for sub-010: 0.27657718273747967
pastient sub-011
Mean FA_NAWM for sub-011: nan
Mean FA_lesion for sub-011: nan
pastient sub-012


/tmp/ipykernel_2170584/3174413930.py:30: RuntimeWarning:

Mean of empty slice



Mean FA_NAWM for sub-012: 0.4020314511579621
Mean FA_lesion for sub-012: 0.39355091413237014
pastient sub-013
Mean FA_NAWM for sub-013: 0.3386019648491474
Mean FA_lesion for sub-013: 0.25194352120161057
pastient sub-014
Mean FA_NAWM for sub-014: 0.2851364785466192
Mean FA_lesion for sub-014: 0.2144465669989586
pastient sub-015
Mean FA_NAWM for sub-015: 0.2930260920669373
Mean FA_lesion for sub-015: nan
pastient sub-016
Mean FA_NAWM for sub-016: nan
Mean FA_lesion for sub-016: nan
pastient sub-017
Mean FA_NAWM for sub-017: 0.3248961715523365
Mean FA_lesion for sub-017: nan
pastient sub-018
Mean FA_NAWM for sub-018: nan
Mean FA_lesion for sub-018: nan
pastient sub-019
Mean FA_NAWM for sub-019: nan
Mean FA_lesion for sub-019: nan
pastient sub-020
Mean FA_NAWM for sub-020: 0.38849598188035017
Mean FA_lesion for sub-020: nan
pastient sub-021
Mean FA_NAWM for sub-021: 0.38418662409251364
Mean FA_lesion for sub-021: nan
pastient sub-022
pastient sub-023
Mean FA_NAWM for sub-023: 0.52336510029

In [80]:
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    
    files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{subj}/ses-M6/anat/*SPAIR2D_T2w*.nii.gz')
    fa_files = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/fa*.nii.gz')
    adc_file = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/adc*.nii.gz')
    print(f"freeview {files_dir1[0]} {fa_files[0]} {adc_file[0]}")

freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-003/ses-M6/anat/sub-003_ses-M6_acq-1901T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-004/ses-M6/anat/sub-004_ses-M6_acq-801T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-005/ses-M6/anat/sub-005_ses-M6_acq-801T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-006/ses-M6/anat/sub-006_ses-M6_acq-1801T2W_SPAIR2D_T2w.nii.gz /NAS/cool

In [ ]:
import glob

SUBJECT = 'sub-091'
print(df_results[df_results['Subject'] == SUBJECT])


# Afficher les fichiers du premier répertoire
files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{SUBJECT}/ses-M6/anat/*SPAIR2D_T2w*.nii.gz')
for file in files_dir1:
    print(file)


# Afficher les fichiers du premier répertoire
files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{SUBJECT}/ses-M3/anat/*SPAIR2D_T2w*.nii.gz')
for file in files_dir1:
    print(file)

# Afficher les fichiers du deuxième répertoire
files_dir2 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJECT}/*.nii.gz')
print("Fichiers dans le deuxième répertoire:")
for file in files_dir2:
    print(file)


    Subject  FA_NAWM  FA_lesion
64  sub-091  0.40518    0.67741
Fichiers dans le premier répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M6/anat/sub-091_ses-M6_acq-1501T2W_SPAIR2D_T2w.nii.gz
Fichiers dans le premier répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_res_RPI_seg.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_recM3onM6_warped.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_RPI_seg.nii.gz
Fichiers dans le deuxième répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/fa_M3onM6.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/lesion_M3_dilated.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/ad_

In [74]:
from scipy.stats import ttest_ind

import plotly.express as px

# Créer un DataFrame avec les données triées
df_FA_NAWM_sorted = df_results.sort_values(by="FA_NAWM")

# Créer un graphique interactif avec Plotly
fig = px.box(
    df_FA_NAWM_sorted.melt(id_vars=["Subject"], value_vars=["FA_NAWM", "FA_lesion"]),
    x="variable",
    y="value",
    color="variable",
    hover_data=["Subject"],
    labels={"value": "FA Values", "variable": "Group"},
    title="Boxplots interactifs de FA_NAWM et FA_lesion"
)

# Afficher le graphique
fig.show()

# Effectuer un test t pour comparer les deux groupes
t_stat, p_value = ttest_ind(fa_nawm_values.dropna(), fa_lesion_values.dropna(), equal_var=False)

# Afficher les résultats du test t
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: -0.5557475448936806, P-value: 0.583426071083629


In [72]:
# Effectuer un test t pour comparer les deux groupes
t_stat, p_value = ttest_ind(fa_nawm_values.dropna(), fa_lesion_values.dropna(), equal_var=False)

# Afficher les résultats du test t
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: -0.5557475448936806, P-value: 0.583426071083629
